In [1]:
import numpy as np
import struct 

def load_mnist_images(filename):
    # Open the file in read-binary mode
    with open(filename, 'rb') as f:
        # Read the magic number (first 4 bytes)
        magic_number = struct.unpack('>I', f.read(4))[0]

        # Check that the file is a 3-dimensional array (magic number should be 2051)
        if magic_number != 2051:
            raise ValueError(f"Invalid magic number {magic_number} in file: {filename}")

        # Read the number of images, number of rows, and number of columns
        num_images = struct.unpack('>I', f.read(4))[0]
        num_rows = struct.unpack('>I', f.read(4))[0]
        num_columns = struct.unpack('>I', f.read(4))[0]

        # Read the rest of the file as a numpy array of unsigned bytes (uint8)
        images = np.frombuffer(f.read(), dtype=np.uint8)

        # Reshape the flat array into a 3D array (num_images, height, width)
        images = images.reshape(num_images, num_rows, num_columns)

        return images
    
def load_mnist_labels(filename):
    # Open the file in read-binary mode
    with open(filename, 'rb') as f:
        # Read the magic number (first 4 bytes)
        magic_number = struct.unpack('>I', f.read(4))[0]

        # Check that the file is a 1-dimensional array (magic number should be 2049)
        if magic_number != 2049:
            raise ValueError(f"Invalid magic number {magic_number} in file: {filename}")

        # Read the number of labels (next 4 bytes)
        num_labels = struct.unpack('>I', f.read(4))[0]

        # Read the rest of the file as a numpy array of unsigned bytes (uint8)
        labels = np.frombuffer(f.read(), dtype=np.uint8)

        return labels

# Example usage:
filename1 = './archive/train-images.idx3-ubyte'
filename2 = './archive/train-labels.idx1-ubyte'
images = load_mnist_images(filename1)
labels = load_mnist_labels(filename2)

print(f"Loaded {images.shape[0]} images with shape {images.shape[1:]} (rows, columns)")
print(f"Loaded {labels.shape[0]} labels")

Loaded 60000 images with shape (28, 28) (rows, columns)
Loaded 60000 labels


In [27]:
def one_hot_encode(labels, num_classes):
    # Create a zero matrix of size (number of labels, number of classes)
    one_hot_encoded = np.zeros((labels.shape[0], num_classes))
    
    # Set the corresponding index to 1 for each label
    one_hot_encoded[np.arange(labels.shape[0]), labels] = 1
    
    return one_hot_encoded

num_classes = 10  # For MNIST, digits range from 0 to 9, so we have 10 classes
one_hot_labels = one_hot_encode(labels, num_classes)
#images_light = images[0:4000, :, :]
#one_hot_labels_light = one_hot_labels[0:4000, :]

print(images.shape)
#print(images_light.shape)
print(labels.shape)
print(one_hot_labels.shape)
#print(one_hot_labels_light.shape)

(60000, 28, 28)
(60000,)
(60000, 10)


In [28]:
from ConvNet_functions import TwoDim_Conv_layer, Relu, make_params, forward_prop, softmax,  cross_entropy_loss, backprop
from ConvNet_model import create_cnn_model, forward_pass, train_cnn, train_cnn_minibatch

input_shape = (6000, 28, 28)  # 1000 images of 28x28 pixels
num_classes = 10
num_filters = 16
kernel_size = 3
stride = 1
num_epochs = 100
learning_rate = 0.001

#images_light_labels = one_hot_labels_light.T # y needs to be in shape (num classes, num examples) to be used in train_cnn(), due to cross entropy loss function

# Train the model
kernels, bias, W, b2 = train_cnn_minibatch(images, one_hot_labels.T , num_epochs, learning_rate, num_filters, kernel_size, stride, batch_size= 32)
print('done')
# Make predictions
#y_pred, _, _, _ = forward_pass(X, kernels, bias, W, b2, stride)
#predicted_classes = np.argmax(y_pred, axis=0)

Epoch 0, Loss: 0.8725562154380216
Epoch 10, Loss: 0.6590677979889269
Epoch 20, Loss: 0.663944810704661
Epoch 30, Loss: 0.7806920558016632
Epoch 40, Loss: 0.7918811773148839
Epoch 50, Loss: 0.7816344997995878
Epoch 60, Loss: 0.7596590268798348
Epoch 70, Loss: 0.7361422155296558
Epoch 80, Loss: 0.700387801400998
Epoch 90, Loss: 0.6428485448306177
done


In [29]:
# Function to evaluate the model on testing data
def evaluate_model(X_test, y_test, kernels, bias, W, b2, stride):
    """
    Evaluates the model on test data.
    
    Args:
    X_test: numpy array of test images (shape: num_examples, height, width)
    y_test: numpy array of one-hot encoded test labels (shape: num_classes, num_examples)
    kernels, bias, W, b2: trained parameters from the CNN
    stride: stride used during the convolution operation
    
    Returns:
    accuracy: The accuracy of the model on the test set
    loss: Cross-entropy loss on the test set
    """
    # Perform a forward pass on the test data
    y_pred, _, _, _ = forward_pass(X_test, kernels, bias, W, b2, stride)
    
    # Compute the cross-entropy loss
    loss = cross_entropy_loss(y_test, y_pred)
    
    # Get the predicted class for each test example
    predicted_classes = np.argmax(y_pred, axis=0)
    
    # Get the true class from y_test
    true_classes = np.argmax(y_test, axis=0)
    
    # Compute the accuracy
    accuracy = np.mean(predicted_classes == true_classes)
    
    print(f"Test Loss: {loss}")
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    
    return loss, accuracy

#Create Testing Data
X_test = images[2000:6000, :, :] #using next 1,000 images for testing
Y_test = one_hot_labels[2000:6000, :]
Y_test_transpose = Y_test.T

#Use Other Test Data
test_file1 = './archive/t10k-images.idx3-ubyte'
test_file2 = './archive/t10k-labels.idx1-ubyte'
X_test_legit = load_mnist_images(test_file1)
Y_test_legit = one_hot_encode(load_mnist_labels(test_file2), 10)
Y_test_legit = Y_test_legit.T

#Evaluate on Testing Data
test_loss, test_accuracy = evaluate_model(X_test_legit, Y_test_legit, kernels, bias, W, b2, stride)


Test Loss: 0.9186965847792408
Test Accuracy: 93.87%
